In [1]:
import requests

In [2]:
class getWeather:

	data = None
	count = None

	def __init__(self, apiKey, city, unit):
		self.apiKey = apiKey
		self.unit = unit
		self.city = city
		
	def requestForecast(self, apiKey, city, unit, cast = 'forecast'):
		url = f'http://api.openweathermap.org/data/2.5/{cast}?appid={apiKey}&q={city}&mode=json&units={unit}' 

		response = requests.get(url).json()
		self.count = response['cnt']
		self.data = response
		
		print('New data has been retrieved')
		
		return response

	def printForecast(self):
		for day in range(self.count):
			date = self.data['list'][day]['dt_txt']
			temp = self.data['list'][day]['main']['temp']
			humidity = self.data['list'][day]['main']['humidity']
			windSpeed = self.data['list'][day]['wind']['speed']
			windDeg = self.data['list'][day]['wind']['deg']
			weather = self.data['list'][day]['weather'][0]['description']
			weatherId = self.data['list'][day]['weather'][0]['id']
			cloud = self.data['list'][day]['clouds']['all']
			
			print('\nDate/time:	{}'.format(date))
			print('Temperature: 	{} Celsius'.format(temp))
			print('Humidity:	{}%'.format(humidity))
			print('windspeed:	{} m/s'.format(windSpeed))
			print('windDeg:	{}'.format(windDeg))
			print('Clouds: 	{}%'.format(cloud))
			print('Weather ID: 	{}'.format(weatherId))
			print('Weather: 	{}'.format(weather))

In [3]:
apiKey = '44a84e47fe4a6d594aab047b7e5e816d'	#change key
unit = 'metric'
#city = input('Enter city: ')
city = 'Arnhem'
cast = 'forecast'

In [4]:
weatherData = getWeather(apiKey, city, unit) #not needed

In [5]:
data = weatherData.requestForecast(apiKey, city, unit) 


New data has been retrieved


In [6]:
weatherData.printForecast()


Date/time:	2021-04-23 09:00:00
Temperature: 	8.17 Celsius
Humidity:	65%
windspeed:	2.43 m/s
windDeg:	5
Clouds: 	90%
Weather ID: 	804
Weather: 	overcast clouds

Date/time:	2021-04-23 12:00:00
Temperature: 	9.7 Celsius
Humidity:	56%
windspeed:	2.68 m/s
windDeg:	15
Clouds: 	63%
Weather ID: 	803
Weather: 	broken clouds

Date/time:	2021-04-23 15:00:00
Temperature: 	11.7 Celsius
Humidity:	49%
windspeed:	2.87 m/s
windDeg:	6
Clouds: 	39%
Weather ID: 	802
Weather: 	scattered clouds

Date/time:	2021-04-23 18:00:00
Temperature: 	9.71 Celsius
Humidity:	65%
windspeed:	3.07 m/s
windDeg:	4
Clouds: 	16%
Weather ID: 	801
Weather: 	few clouds

Date/time:	2021-04-23 21:00:00
Temperature: 	5.08 Celsius
Humidity:	79%
windspeed:	3.83 m/s
windDeg:	26
Clouds: 	29%
Weather ID: 	802
Weather: 	scattered clouds

Date/time:	2021-04-24 00:00:00
Temperature: 	2.76 Celsius
Humidity:	89%
windspeed:	2.68 m/s
windDeg:	49
Clouds: 	53%
Weather ID: 	803
Weather: 	broken clouds

Date/time:	2021-04-24 03:00:00
Temperature: 

In [7]:
from fastai.collab import *
from fastai.tabular.all import *

In [8]:
forecast_raw = pd.DataFrame(columns = ['date_time','temp','humid','wspd','wdeg','cloud','weather_id'])

In [11]:
number_of_rows = data['cnt']

for i in range(number_of_rows):
    date = data['list'][i]['dt_txt']
    temp = data['list'][i]['main']['temp']
    humidity = data['list'][i]['main']['humidity']
    windSpeed = data['list'][i]['wind']['speed']
    windDeg = data['list'][i]['wind']['deg']
    weatherId = data['list'][i]['weather'][0]['id']
    cloud = data['list'][i]['clouds']['all']
    
    forecast_raw = forecast_raw.append({
        'date_time':date,
        'temp':temp,
        'humid':humidity,
        'wspd':windSpeed,
        'wdeg':windDeg,
        'cloud':cloud,
        'weather_id':weatherId
    }, ignore_index=True)

In [12]:
forecast_raw.head()

,date_time,temp,humid,wspd,wdeg,cloud,weather_id
0,2021-04-23 09:00:00,8.17,65,2.43,5,90,804
1,2021-04-23 12:00:00,9.70,56,2.68,15,63,803
2,2021-04-23 15:00:00,11.70,49,2.87,6,39,802
3,2021-04-23 18:00:00,9.71,65,3.07,4,16,801
4,2021-04-23 21:00:00,5.08,79,3.83,26,29,802


In [13]:
#remove data at night
for i in range(len(forecast_raw)):
    if int(forecast_raw['date_time'][i][11:13]) < 4 or int(forecast_raw['date_time'][i][11:13]) > 22: 
        forecast_raw.drop(index=i, axis=0, inplace=True)

In [14]:
forecast_raw.reset_index(inplace=True)
forecast_raw.head()

,index,date_time,temp,humid,wspd,wdeg,cloud,weather_id
0,0,2021-04-23 09:00:00,8.17,65,2.43,5,90,804
1,1,2021-04-23 12:00:00,9.70,56,2.68,15,63,803
2,2,2021-04-23 15:00:00,11.70,49,2.87,6,39,802
3,3,2021-04-23 18:00:00,9.71,65,3.07,4,16,801
4,4,2021-04-23 21:00:00,5.08,79,3.83,26,29,802


In [15]:
#To find the value with the most apperance
from collections import Counter

def most_frequent(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

In [20]:
def merge_data(weather):
    new_data = pd.DataFrame(columns = ['date',
                                       'mor_temp','mor_humid','mor_wspd','mor_wdeg','mor_cloud','mor_weather_id',
                                       'noo_temp','noo_humid','noo_wspd','noo_wdeg','noo_cloud','noo_weather_id',
                                       'aft_temp','aft_humid','aft_wspd','aft_wdeg','aft_cloud','aft_weather_id',])
    
    mor_count, mor_temp, mor_humid, mor_wspd, mor_wdeg, mor_cloud = [0]*6
    noo_count, noo_temp, noo_humid, noo_wspd, noo_wdeg, noo_cloud = [0]*6
    aft_count, aft_temp, aft_humid, aft_wspd, aft_wdeg, aft_cloud = [0]*6
    
    mor_weather_id=[]
    noo_weather_id=[]
    aft_weather_id=[]
    
    current_date = weather.loc[0, 'date_time'][0:10]
    
    for i in range(len(weather)):
        if weather.loc[i, 'date_time'][0:10] != current_date:
            new_data = new_data.append({
                'date':current_date,
                'mor_temp':mor_temp/mor_count,
                'mor_humid':mor_humid/mor_count,
                'mor_wspd':mor_wspd/mor_count,
                'mor_wdeg':mor_wdeg/mor_count,
                'mor_cloud':mor_cloud/mor_count,
                'mor_weather_id':most_frequent(mor_weather_id),
                'noo_temp':noo_temp/noo_count,
                'noo_humid':noo_humid/noo_count,
                'noo_wspd':noo_wspd/noo_count,
                'noo_wdeg':noo_wdeg/noo_count,
                'noo_cloud':noo_cloud/noo_count,
                'noo_weather_id':most_frequent(noo_weather_id),
                'aft_temp':aft_temp/aft_count,
                'aft_humid':aft_humid/aft_count,
                'aft_wspd':aft_wspd/aft_count,
                'aft_wdeg':aft_wdeg/aft_count,
                'aft_cloud':aft_cloud/aft_count,
                'aft_weather_id':most_frequent(aft_weather_id)
            }, ignore_index=True)
            
            mor_count, mor_temp, mor_humid, mor_wspd, mor_wdeg, mor_cloud = [0]*6
            noo_count, noo_temp, noo_humid, noo_wspd, noo_wdeg, noo_cloud = [0]*6
            aft_count, aft_temp, aft_humid, aft_wspd, aft_wdeg, aft_cloud = [0]*6
            
            mor_weather_id=[]
            noo_weather_id=[]
            aft_weather_id=[]
    
            current_date = weather.loc[i, 'date_time'][0:10]
        
        if int(weather.loc[i, 'date_time'][11:13]) < 11:
            mor_count += 1
            mor_temp += weather.loc[i, 'temp']
            mor_humid += weather.loc[i, 'humid']
            mor_wspd += weather.loc[i, 'wspd']
            mor_wdeg += weather.loc[i, 'wdeg']
            mor_cloud += weather.loc[i, 'cloud']
            mor_weather_id.append(weather['weather_id'][i])
            
        elif int(weather.loc[i, 'date_time'][11:13]) < 16:
            noo_count += 1
            noo_temp += weather.loc[i, 'temp']
            noo_humid += weather.loc[i, 'humid']
            noo_wspd += weather.loc[i, 'wspd']
            noo_wdeg += weather.loc[i, 'wdeg']
            noo_cloud += weather.loc[i, 'cloud']
            noo_weather_id.append(weather['weather_id'][i])
            
        else:
            aft_count += 1
            aft_temp += weather.loc[i, 'temp']
            aft_humid += weather.loc[i, 'humid']
            aft_wspd += weather.loc[i, 'wspd']
            aft_wdeg += weather.loc[i, 'wdeg']
            aft_cloud += weather.loc[i, 'cloud']
            aft_weather_id.append(weather['weather_id'][i])
        
    return new_data

In [21]:
forecast_data = merge_data(forecast_raw)

In [22]:
forecast_data.head()

,date,mor_temp,mor_humid,mor_wspd,mor_wdeg,mor_cloud,mor_weather_id,noo_temp,noo_humid,noo_wspd,noo_wdeg,noo_cloud,noo_weather_id,aft_temp,aft_humid,aft_wspd,aft_wdeg,aft_cloud,aft_weather_id
0,2021-04-23,8.170,65.0,2.430,5.0,90.0,804,10.700,52.5,2.775,10.5,51.0,803,7.395,72.0,3.450,15.0,22.5,801
1,2021-04-24,6.660,71.5,2.860,39.0,99.5,804,12.295,51.0,4.470,26.5,99.0,804,6.065,71.0,4.650,21.5,65.5,804
2,2021-04-25,5.345,68.0,3.565,36.0,13.0,800,12.190,44.5,4.155,32.0,58.5,801,6.105,71.5,4.175,22.0,36.5,803
3,2021-04-26,5.610,68.5,3.160,39.0,9.0,800,12.155,45.0,4.240,47.5,49.0,801,7.150,71.0,3.935,34.0,96.5,804
4,2021-04-27,7.670,67.0,3.855,52.5,100.0,804,14.170,44.0,4.425,62.5,100.0,804,9.240,64.0,4.195,40.0,97.5,804


In [25]:
forecast_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            5 non-null      object 
 1   mor_temp        5 non-null      float64
 2   mor_humid       5 non-null      float64
 3   mor_wspd        5 non-null      float64
 4   mor_wdeg        5 non-null      float64
 5   mor_cloud       5 non-null      float64
 6   mor_weather_id  5 non-null      object 
 7   noo_temp        5 non-null      float64
 8   noo_humid       5 non-null      float64
 9   noo_wspd        5 non-null      float64
 10  noo_wdeg        5 non-null      float64
 11  noo_cloud       5 non-null      float64
 12  noo_weather_id  5 non-null      object 
 13  aft_temp        5 non-null      float64
 14  aft_humid       5 non-null      float64
 15  aft_wspd        5 non-null      float64
 16  aft_wdeg        5 non-null      float64
 17  aft_cloud       5 non-null      float64

In [26]:
path = Path('.')

In [28]:
path.ls()

(#31) [Path('.~lock.OpenWeather_Antwerp.csv#'),Path('training_data_v3.csv'),Path('training_data_v3_1.csv'),Path('training_data_v2.csv'),Path('Timeseries_52.012_6.131_SA_0deg_-179deg_2005_2016_Doesburg.csv'),Path('.git'),Path('SolarEnergyDataRefine.ipynb'),Path('EnergyProduction_LinearRegression_v2.ipynb'),Path('SolarEnergyRefine_v2.ipynb'),Path('SolarEnergyDataRefine_v3.ipynb')...]

In [32]:
learn = load_learner(path/'export_v2.pkl')

In [33]:
day_list = []
month_list = []
for i in range(len(forecast_data)):
    day_list.append(int(forecast_data.date[i][8:]))
    month_list.append(int(forecast_data.date[i][5:7]))
forecast_data['day'] = day_list
forecast_data['month'] = month_list

In [35]:
forecast_data.drop(['date'], axis=1, inplace=True)
forecast_data.head()

,mor_temp,mor_humid,mor_wspd,mor_wdeg,mor_cloud,mor_weather_id,noo_temp,noo_humid,noo_wspd,noo_wdeg,noo_cloud,noo_weather_id,aft_temp,aft_humid,aft_wspd,aft_wdeg,aft_cloud,aft_weather_id,day,month
0,8.170,65.0,2.430,5.0,90.0,804,10.700,52.5,2.775,10.5,51.0,803,7.395,72.0,3.450,15.0,22.5,801,23,4
1,6.660,71.5,2.860,39.0,99.5,804,12.295,51.0,4.470,26.5,99.0,804,6.065,71.0,4.650,21.5,65.5,804,24,4
2,5.345,68.0,3.565,36.0,13.0,800,12.190,44.5,4.155,32.0,58.5,801,6.105,71.5,4.175,22.0,36.5,803,25,4
3,5.610,68.5,3.160,39.0,9.0,800,12.155,45.0,4.240,47.5,49.0,801,7.150,71.0,3.935,34.0,96.5,804,26,4
4,7.670,67.0,3.855,52.5,100.0,804,14.170,44.0,4.425,62.5,100.0,804,9.240,64.0,4.195,40.0,97.5,804,27,4


In [37]:
predicted_value = []

for i in range(len(forecast_data)):
    row, clas, probs = learn.predict(forecast_data.iloc[i])
    predicted_value.append(clas)

In [38]:
predicted_value

[tensor([15.9693]),
 tensor([16.6496]),
 tensor([13.1155]),
 tensor([9.0641]),
 tensor([12.8652])]